# 9th Question

### Which player has won the mos tournaments in a single month?

Installing needed packages

In [1]:
%pip install numpy pandas pyarrow

Note: you may need to restart the kernel to use updated packages.


Importing needed packages

In [2]:
import pandas as pd
import numpy as np
from glob import glob
import os

# Reading .parquet data
According to question and ERD in /data/ERD folder as Tennis Schema.pdf , We are analyzing the torunaments data
It might take few seconds to read data

In [3]:
days_to_query = os.listdir("../data/raw/tennis_data")
files_to_load = []

for day in days_to_query:
    files_to_load.extend(
        glob(
            f"../data/raw/tennis_data/{day}/data/raw/raw_match_parquet*/tournament_*.parquet"
        )
    )

tournament_data_frame = pd.concat(
    [
        pd.read_parquet(
            file,
            columns=[
                "match_id",
                "tournament_name"
            ],
        )
        for file in files_to_load
    ],
    ignore_index=True,
)

### Reading data of Statistics

In [4]:
days_to_query = os.listdir("../data/raw/tennis_data")
files_to_load = []

for day in days_to_query:
    files_to_load.extend(
        glob(
            f"../data/raw/tennis_data/{day}/data/raw/raw_statistics*/statistics_*.parquet"
        )
    )

tennis_statistics_data_frame = pd.concat(
    [
        pd.read_parquet(
            file,
            columns=[
                "match_id",
                "period",
                "statistic_name",
                "home_stat",
                "away_stat",
                "home_total",
                "away_total",
            ],
        )
        for file in files_to_load
    ],
    ignore_index=True,
)

### Reading data of Matches and Teams

In [5]:
days_to_query_ = os.listdir("../data/raw/tennis_data")
files_to_load_ = []

for day in days_to_query_:
    files_to_load_.extend(
        glob(f"../data/raw/tennis_data/{day}/data/raw/raw_match*/home_team_1*.parquet")
    )

home_statistics_data_frame = pd.concat(
    [
        pd.read_parquet(
            file,
            columns=["match_id", "name"],
        )
        for file in files_to_load_
    ],
    ignore_index=True,
)

In [6]:
days_to_query__ = os.listdir("../data/raw/tennis_data")
files_to_load__ = []

for day in days_to_query__:
    files_to_load__.extend(
        glob(f"../data/raw/tennis_data/{day}/data/raw/raw_match*/away_team_1*.parquet")
    )

away_statistics_data_frame = pd.concat(
    [
        pd.read_parquet(
            file,
            columns=["match_id", "name"],
        )
        for file in files_to_load__
    ],
    ignore_index=True,
)

### Reading event data

In [13]:
days_to_query__ = os.listdir("../data/raw/tennis_data")
files_to_load__ = []

for day in days_to_query__:
    files_to_load__.extend(
        glob(f"../data/raw/tennis_data/{day}/data/raw/raw_match*/event*.parquet")
    )

event_data_frame = pd.concat(
    [
        pd.read_parquet(
            file,
            columns=["start_datetime"],
        )
        for file in files_to_load__
    ],
    ignore_index=True,
)

Display data of home matches required information :

In [7]:
home_statistics_data_frame.rename(
    columns={"name": "Home Player", "match_id": "Match_id"}, inplace=True
)
home_statistics_data_frame.head()

,Match_id,Home Player
0,11998445,Cazaux A.
1,11998446,Lestienne C.
2,11998447,Ćorić B.
3,11998448,Mmoh M.
4,11998449,Paire B.


Display data of away matches required information

In [8]:
away_statistics_data_frame.rename(
    columns={"name": "Away Player", "match_id": "Match_id"}, inplace=True
)
away_statistics_data_frame.head()

,Match_id,Away Player
0,11998445,Auger-Aliassime F.
1,11998446,Cobolli F.
2,11998447,Martínez P.
3,11998448,Muller A.
4,11998449,Mayot H.


Merging whole data

In [9]:
merged_df = pd.merge(
    home_statistics_data_frame, away_statistics_data_frame, on="Match_id", how="inner"
)
merged_df.rename(
    columns={"Home_Player_Name": "Home_Player", "Away_Player_Name": "Away_Player"},
    inplace=True,
)
merged_df_unique = merged_df.drop_duplicates(subset=["Match_id"], keep="last")
merged_df_unique.head(10)

,Match_id,Home Player,Away Player
6,11998448,Mmoh M.,Muller A.
7,11998449,Paire B.,Mayot H.
8,11998450,Shapovalov D.,Bublik A.
9,11998451,Shevchenko A.,Barrere G.
23,11998672,Imai S.,Sekulic P.
24,11998674,Saville L.,Jones H.
25,11998675,Hijikata R.,Jasika O.
26,11998676,Noguchi R.,Sweeny D.
29,11998774,Duckworth J.,Maloney P.
32,11998782,Sandgren T.,Fanselow S.


In [10]:
filtered_df = tennis_statistics_data_frame[
    (tennis_statistics_data_frame["period"] == "ALL")
    & (tennis_statistics_data_frame["statistic_name"] == "total")
]
filtered_df.rename(columns={"match_id": "Match_id"}, inplace=True)
filtered_df.drop_duplicates(subset=["Match_id"], keep="first", inplace=True)
filtered_df.head(10)

C:\Users\Shahab\AppData\Local\Temp\ipykernel_29756\1009194557.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.rename(columns={"match_id": "Match_id"}, inplace=True)
C:\Users\Shahab\AppData\Local\Temp\ipykernel_29756\1009194557.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.drop_duplicates(subset=["Match_id"], keep="first", inplace=True)


,Match_id,period,statistic_name,home_stat,away_stat,home_total,away_total
8,11998445,ALL,total,95,96,NaN,NaN
79,11998446,ALL,total,46,63,NaN,NaN
133,11998447,ALL,total,66,48,NaN,NaN
186,11998448,ALL,total,63,48,NaN,NaN
240,11998449,ALL,total,47,62,NaN,NaN
294,11998450,ALL,total,100,107,NaN,NaN
364,11998451,ALL,total,94,92,NaN,NaN
435,11998456,ALL,total,90,103,NaN,NaN
506,11998459,ALL,total,70,58,NaN,NaN
560,11998666,ALL,total,72,80,NaN,NaN


In [11]:
final_df = pd.merge(merged_df_unique, filtered_df, on="Match_id", how="inner")
final_df.rename(
    columns={"Home_Player_Name": "Home_Player", "Away_Player_Name": "Away_Player"},
    inplace=True,
)
final_df.head(25)

,Match_id,Home Player,Away Player,period,statistic_name,home_stat,away_stat,home_total,away_total
0,11998448,Mmoh M.,Muller A.,ALL,total,63,48,NaN,NaN
1,11998449,Paire B.,Mayot H.,ALL,total,47,62,NaN,NaN
2,11998450,Shapovalov D.,Bublik A.,ALL,total,100,107,NaN,NaN
3,11998451,Shevchenko A.,Barrere G.,ALL,total,94,92,NaN,NaN
4,11998672,Imai S.,Sekulic P.,ALL,total,126,112,NaN,NaN
5,11998674,Saville L.,Jones H.,ALL,total,111,95,NaN,NaN
6,11998675,Hijikata R.,Jasika O.,ALL,total,106,111,NaN,NaN
7,11998676,Noguchi R.,Sweeny D.,ALL,total,59,76,NaN,NaN
8,11998774,Duckworth J.,Maloney P.,ALL,total,65,45,NaN,NaN
9,11998782,Sandgren T.,Fanselow S.,ALL,total,73,54,NaN,NaN


In [12]:
final_df.rename(
    columns={"home_stat": "Home Score", "away_stat": "Away Score"}, inplace=True
)
final_df = final_df.drop(
    columns=["period", "statistic_name", "home_total", "away_total"]
)
final_df.head(25)

,Match_id,Home Player,Away Player,Home Score,Away Score
0,11998448,Mmoh M.,Muller A.,63,48
1,11998449,Paire B.,Mayot H.,47,62
2,11998450,Shapovalov D.,Bublik A.,100,107
3,11998451,Shevchenko A.,Barrere G.,94,92
4,11998672,Imai S.,Sekulic P.,126,112
5,11998674,Saville L.,Jones H.,111,95
6,11998675,Hijikata R.,Jasika O.,106,111
7,11998676,Noguchi R.,Sweeny D.,59,76
8,11998774,Duckworth J.,Maloney P.,65,45
9,11998782,Sandgren T.,Fanselow S.,73,54


# Data was corrupt and I don't have idea what to code 
# Thanks in advance